In [1]:
# dataset: https://www.kaggle.com/datasets/jangedoo/utkface-new

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

 99% 327M/331M [00:08<00:00, 44.2MB/s]
100% 331M/331M [00:08<00:00, 38.8MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
len(age)

23708

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [9]:
df.shape

(23708, 3)

In [10]:
df.head()

,age,gender,img
0,1,0,1_0_2_20161219194800523.jpg.chip.jpg
1,25,1,25_1_2_20170116175319780.jpg.chip.jpg
2,27,1,27_1_2_20170116163552814.jpg.chip.jpg
3,37,1,37_1_2_20170116184911351.jpg.chip.jpg
4,29,1,29_1_0_20170104165932577.jpg.chip.jpg


In [11]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [12]:
train_df.shape

(20000, 3)

In [13]:
test_df.shape

(3708, 3)

In [14]:
# doing data augumentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
# a small change is prev y_col is a single feature now its a list

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [16]:
# using resnet as pretrain model
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [17]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 3s 0us/step


In [18]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [19]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [20]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 263s 404ms/step - loss: 103.2034 - age_loss: 15.5037 - gender_loss: 0.8859 - age_mae: 15.5037 - gender_accuracy: 0.5137 - val_loss: 83.2200 - val_age_loss: 14.6852 - val_gender_loss: 0.6923 - val_age_mae: 14.6852 - val_gender_accuracy: 0.5232
Epoch 2/10
625/625 [==============================] - 254s 406ms/step - loss: 83.9171 - age_loss: 15.0606 - gender_loss: 0.6955 - age_mae: 15.0606 - gender_accuracy: 0.5220 - val_loss: 82.9113 - val_age_loss: 14.4030 - val_gender_loss: 0.6920 - val_age_mae: 14.4030 - val_gender_accuracy: 0.5237
Epoch 3/10
625/625 [==============================] - 284s 455ms/step - loss: 83.5829 - age_loss: 15.0405 - gender_loss: 0.6923 - age_mae: 15.0405 - gender_accuracy: 0.5225 - val_loss: 83.5019 - val_age_loss: 14.9840 - val_gender_loss: 0.6921 - val_age_mae: 14.9840 - val_gender_accuracy: 0.5232
Epoch 4/10
625/625 [==============================] - 256s 409ms/step - loss: 83.5136 - age_loss: 14.8442 - gen